<a href="https://colab.research.google.com/github/dfridland/NLP/blob/HW8/NLP_DF_HW8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Урок 8. Рекуррентные нейронные сети RNN, LSTM, GRU

## Домашнее задание.

Данные берем `'отзывы за лето'`

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи

1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN и/или (RNN -> CNN)
4. сделать выводы что получилось лучше

In [ ]:
!pip install stop_words
!pip install pymorphy2

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

df_train = pd.read_csv("./lection8/data/train.csv")
df_test = pd.read_csv("./lection8/data/test.csv")
df_val = pd.read_csv("./lection8/data/val.csv")

In [ ]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw] #долгая Каноническая форма слова (например, форма единственного числа, именительного падежа для существительных)
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np
#import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, Conv2D, GlobalMaxPool1D, concatenate, Flatten, add, MaxPool1D, RepeatVector
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Masking, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, TimeDistributed, AveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.utils.data_utils import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [ ]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train) #Текст в последовательности
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length) #Эта функция преобразует список (длиной num_samples) последовательностей (списков целых чисел) в двумерный массив Numpy 
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [ ]:
y_train = df_train['class'].values
y_val = df_val['class'].values

### RNN Model

In [ ]:
# tf.config.set_visible_devices([], 'GPU')
# with tf.device('/cpu:0'):
physical_devices = tf.config.list_physical_devices('GPU'); 
tf.config.set_visible_devices(physical_devices[0], 'GPU')

In [ ]:
model_name = 'RNN'
model = Sequential()
model.add(
    Embedding(input_dim=word_count, #это размер словаря в текстовых данных
              input_length=training_length, #это длина входных последовательностей, которую вы бы определили для любого входного слоя модели Keras
              output_dim=30, #это размер векторного пространства, в которое будут встроены слова. Он определяет размер выходных векторов из этого слоя для каждого слова. Например, это может быть 32 или 100 или даже больше.
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) #Маскирует последовательность,используя значение маски для пропуска таймфреймов
model.add(SimpleRNN(32, recurrent_dropout=0.2, return_sequences=True)) #возвращать всю последовательность выходных данных для каждого элемента (по одному вектору на каждый шаг)
model.add(SimpleRNN(32, recurrent_dropout=0.2))#отбрасывание линейных значений повторов 0.2
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss') #Класс EarlyStopping Остановить обучение, когда отслеживаемая метрика перестанет улучшаться 


tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+ model_name, 
    write_graph=False, update_freq=100, profile_batch=0)

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping]) 

Epoch 1/10
319/319 [==============================] - 502s 2s/step - loss: 0.6820 - accuracy: 0.5402 - val_loss: 0.5778 - val_accuracy: 0.6948
Epoch 2/10
319/319 [==============================] - 361s 1s/step - loss: 0.4620 - accuracy: 0.7848 - val_loss: 0.5214 - val_accuracy: 0.7385
Epoch 3/10
319/319 [==============================] - 364s 1s/step - loss: 0.1892 - accuracy: 0.9280 - val_loss: 0.6574 - val_accuracy: 0.7267


In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 5s 104ms/step - loss: 0.6659 - accuracy: 0.7226


Test score: 0.6658899784088135
Test accuracy: 0.722611665725708


### CNN -> RNN Model


In [ ]:
model_name = 'CNN -> RNN'
model = Sequential()

model.add(
    Embedding(input_dim=word_count, #это размер словаря в текстовых данных
              input_length=training_length, #это длина входных последовательностей, которую вы бы определили для любого входного слоя модели Keras
              output_dim=64, #это размер векторного пространства, в которое будут встроены слова. Он определяет размер выходных векторов из этого слоя для каждого слова. Например, это может быть 32 или 100 или даже больше.
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) #Маскирует последовательность,используя значение маски для пропуска таймфреймов
model.add(Conv1D(128, 3, activation='relu', padding="same"))
model.add(SimpleRNN(64, recurrent_dropout=0.2)) #отбрасывание линейных значений повторов 0.2
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss') #Класс EarlyStopping Остановить обучение, когда отслеживаемая метрика перестанет улучшаться 

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+ model_name, 
    write_graph=False, update_freq=100, profile_batch=0)



In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
319/319 [==============================] - 138s 428ms/step - loss: 0.5907 - accuracy: 0.6635 - val_loss: 0.5044 - val_accuracy: 0.7459
Epoch 2/10
319/319 [==============================] - 131s 409ms/step - loss: 0.3405 - accuracy: 0.8556 - val_loss: 0.5371 - val_accuracy: 0.7465


### RNN -> CNN Model

In [ ]:
model_name = 'RNN -> CNN'
model = Sequential()

model.add(
    Embedding(input_dim=word_count, #это размер словаря в текстовых данных
              input_length=training_length, #это длина входных последовательностей, которую вы бы определили для любого входного слоя модели Keras
              output_dim=64, #это размер векторного пространства, в которое будут встроены слова. Он определяет размер выходных векторов из этого слоя для каждого слова. Например, это может быть 32 или 100 или даже больше.
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) #Маскирует последовательность,используя значение маски для пропуска таймфреймов
model.add(SimpleRNN(64, recurrent_dropout=0.2, return_sequences=True)) #отбрасывание линейных значений повторов 0.2
# model.add(RepeatVector(32))
model.add(Conv1D(128, 3, activation='relu', padding="same"))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss') #Класс EarlyStopping Остановить обучение, когда отслеживаемая метрика перестанет улучшаться 

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+ model_name, 
    write_graph=False, update_freq=100, profile_batch=0)


In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
39/39 [==============================] - 7s 132ms/step - loss: 0.6950 - accuracy: 0.5092 - val_loss: nan - val_accuracy: 0.4934
Epoch 2/10
39/39 [==============================] - 4s 114ms/step - loss: 0.6920 - accuracy: 0.5179 - val_loss: nan - val_accuracy: 0.5207


# LSTM Models

In [ ]:

%%time
# Run inference on CPU

with tf.device('/cpu:0'):
    model_name = 'LSTM'
    model = Sequential()

    model.add(
        Embedding(input_dim=word_count, #это размер словаря в текстовых данных
              input_length=training_length, #это длина входных последовательностей, которую вы бы определили для любого входного слоя модели Keras
              output_dim=30, #это размер векторного пространства, в которое будут встроены слова. Он определяет размер выходных векторов из этого слоя для каждого слова. Например, это может быть 32 или 100 или даже больше.
              trainable=True,
              mask_zero=True))
    model.add(Masking(mask_value=0.0)) #Маскирует последовательность,используя значение маски для пропуска таймфреймов
    model.add(LSTM(64, recurrent_dropout=0.2)) #отбрасывание линейных значений повторов 0.2
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    early_stopping=EarlyStopping(monitor='val_loss') #Класс EarlyStopping Остановить обучение, когда отслеживаемая метрика перестанет улучшаться 

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir='logs/'+ model_name, 
        write_graph=False, update_freq=100, profile_batch=0)

    history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
319/319 [==============================] - 25s 77ms/step - loss: 0.5576 - accuracy: 0.7066 - val_loss: 0.4966 - val_accuracy: 0.7540
Epoch 2/10
319/319 [==============================] - 25s 78ms/step - loss: 0.3303 - accuracy: 0.8639 - val_loss: 0.5281 - val_accuracy: 0.7498
CPU times: user 1min 32s, sys: 46.7 s, total: 2min 19s
Wall time: 50.4 s


In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 6s 123ms/step - loss: 0.5650 - accuracy: 0.7384


Test score: 0.5649610757827759
Test accuracy: 0.7383943796157837


In [ ]:
df_test.text.values

array(['тектоника рельефсамый ужасный мир мучение',
       'ходить запускать шар желание насна получиться хрен они',
       'хотеть лето ради направить ноготь яркий лак', ...,
       'rt killgayslut yournovocaine привеееть муд черта сэмми',
       'настроение вроде нормальный плакать хотеться',
       'зайти сон девчонкампока мыть посудунастя фоткаться httptcokinexudtuh'],
      dtype=object)

In [ ]:
def preprocess_text_for_prediction(txt):
  txt = preprocess_text(txt)
  txt = tokenizer.texts_to_sequences([txt])
  matr_txt = pad_sequences(txt, maxlen=training_length)
  mart_txt = matr_txt.reshape(1, -1)
  return matr_txt


In [ ]:
model.predict(
    [preprocess_text_for_prediction('тектоника рельефсамый ужасный мир мучение')],
    batch_size=None,
    )

1/1 [==============================] - 2s 2s/step


array([[0.12535053]], dtype=float32)

In [ ]:
model.predict(
    [preprocess_text_for_prediction('Люблю смотреть на звезды')],
    batch_size=None,
)

1/1 [==============================] - 0s 156ms/step


array([[0.84056103]], dtype=float32)

In [ ]:
model.predict(
    [preprocess_text_for_prediction('Великолепно!')], 
    batch_size=None,
)

1/1 [==============================] - 0s 135ms/step


array([[0.808098]], dtype=float32)

### Bidirectional LSTM

In [ ]:
%%time
# Run inference on CPU

with tf.device('/cpu:0'):
    model_name = 'Bidirectional LSTM'
    inputs = Input(shape=(X_train.shape[1],))

    x =     Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True)(inputs)

    xbi = Bidirectional(LSTM(15, return_sequences=True))(x) #Двунаправленный
    x = add([x, xbi])

    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1, activation='sigmoid')(x)



    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

CPU times: user 534 ms, sys: 85.4 ms, total: 619 ms
Wall time: 513 ms


In [ ]:
%%time
# Run inference on CPU

with tf.device('/cpu:0'):
    early_stopping=EarlyStopping(monitor='val_loss')  

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir='logs/'+ model_name, 
        write_graph=False, update_freq=100, profile_batch=0)


    history_1 = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
319/319 [==============================] - 11s 28ms/step - loss: 0.5508 - accuracy: 0.7086 - val_loss: 0.4873 - val_accuracy: 0.7590
Epoch 2/10
319/319 [==============================] - 9s 27ms/step - loss: 0.2985 - accuracy: 0.8779 - val_loss: 0.5381 - val_accuracy: 0.7460
CPU times: user 51.9 s, sys: 17.5 s, total: 1min 9s
Wall time: 19.6 s


In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 8s 167ms/step - loss: 0.5663 - accuracy: 0.7343


Test score: 0.5663064122200012
Test accuracy: 0.7343384623527527


#CNN Model


In [ ]:
%%time
# Run training on GPU
with tf.device('/gpu:0'):
    model_name = 'CNN'
    model = Sequential()
    model.add(
        Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=100,
              trainable=True,
              mask_zero=True))
#model.add(Masking(mask_value=0.0))
    model.add(Conv1D(128, 3))
    model.add(Activation("relu"))
    model.add(GlobalMaxPool1D())
    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(
        optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

CPU times: user 51 ms, sys: 63 ms, total: 114 ms
Wall time: 138 ms


In [ ]:
%%time
# Run training on GPU
with tf.device('/gpu:0'):
    early_stopping=EarlyStopping(monitor='val_loss')  

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir='logs/'+ model_name, 
        write_graph=False, update_freq=100, profile_batch=0)

    history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
319/319 [==============================] - 23s 68ms/step - loss: 0.5369 - accuracy: 0.7159 - val_loss: 0.4810 - val_accuracy: 0.7571
Epoch 2/10
319/319 [==============================] - 19s 61ms/step - loss: 0.2505 - accuracy: 0.8990 - val_loss: 0.5483 - val_accuracy: 0.7451
CPU times: user 13.6 s, sys: 5.36 s, total: 19 s
Wall time: 42.1 s


In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 9ms/step - loss: 0.5875 - accuracy: 0.7368


Test score: 0.58748859167099
Test accuracy: 0.7367632389068604


# CNN -> LSTM

In [ ]:
%%time
# Run inference on CPU

with tf.device('/cpu:0'):
    model_name = 'CNN'
    model = Sequential()
    model.add(
        Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
    model.add(Masking(mask_value=0.0))
    model.add(Conv1D(128, 3, activation='relu', padding="same"))
    model.add(LSTM(64, recurrent_dropout=0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    early_stopping=EarlyStopping(monitor='val_loss')  

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir='logs/'+ model_name, 
        write_graph=False, update_freq=100, profile_batch=0)

    history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
319/319 [==============================] - 33s 101ms/step - loss: 0.5504 - accuracy: 0.7096 - val_loss: 0.4883 - val_accuracy: 0.7545
Epoch 2/10
319/319 [==============================] - 34s 107ms/step - loss: 0.2989 - accuracy: 0.8762 - val_loss: 0.5355 - val_accuracy: 0.7426
CPU times: user 2min 21s, sys: 1min 15s, total: 3min 37s
Wall time: 1min 7s


In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 5s 96ms/step - loss: 0.5637 - accuracy: 0.7397


Test score: 0.563737690448761
Test accuracy: 0.7396728992462158


# LSTM -> CNN

In [ ]:
%%time
# Run inference on CPU

with tf.device('/cpu:0'):
    
    model_name = 'LSTM -> CNN'
    model = Sequential()
    model.add(
        Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
    model.add(Masking(mask_value=0.0))
    model.add(LSTM(64, return_sequences=True))
    model.add(Conv1D(64, 3, activation='relu', padding="same"))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    early_stopping=EarlyStopping(monitor='val_loss')  

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir='logs/'+ model_name, 
        write_graph=False, update_freq=100, profile_batch=0)

    history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
319/319 [==============================] - 24s 73ms/step - loss: 0.5565 - accuracy: 0.7029 - val_loss: 0.4951 - val_accuracy: 0.7506
Epoch 2/10
319/319 [==============================] - 26s 80ms/step - loss: 0.3177 - accuracy: 0.8700 - val_loss: 0.5326 - val_accuracy: 0.7451
CPU times: user 1min 43s, sys: 1min 1s, total: 2min 44s
Wall time: 50.4 s


In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 25ms/step - loss: 0.5641 - accuracy: 0.7379


Test score: 0.5641102194786072
Test accuracy: 0.7379094362258911


Similar results on each model. Just RNN is slighrly better.